In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Load Dataset
df = pd.read_csv('Vehicle Risk Prediction Dataset.csv')

In [3]:
# Drop unnecessary features
df.drop(['Vehicle_Speed_Ratio', 'Light_Conditions', 'Medical_Condition', 'Landscape', 'Road_Surface_Conditions'], axis=1, inplace=True)


In [4]:
# Prepare Data for Model
X = df.drop(['Risk_Score'], axis=1)
y = df['Risk_Score']

In [23]:
df.head()

,Driver_Age,Driver_Experience,Driver_Awake_Time,Last_Service_Months_Ago,Cargo_Load,Visibility,Weather,Road_Type,Traffic_Density,Road_Hazards,Time_of_Day,Temperature,Fatigue_Level,Speeding,Risk_Score
0,19,1,5.5,1,88,Good,Clear,Rural,Low,NaN,Morning,22,Low,No,10.4
1,19,1,5.5,1,88,Good,Clear,Rural,Low,NaN,Morning,22,Low,No,10.4
2,19,1,5.5,1,88,Good,Clear,Rural,Low,NaN,Morning,22,Low,No,10.4
3,32,3,5.9,6,167,Good,Clear,Urban,Low,NaN,Afternoon,25,Low,No,14.7
4,32,3,5.9,6,167,Good,Clear,Urban,Low,NaN,Afternoon,25,Low,No,14.7


In [5]:
# Identify numeric and categorical features
numeric_features = ['Driver_Age', 'Driver_Experience', 'Driver_Awake_Time',
                    'Last_Service_Months_Ago', 'Cargo_Load', 'Temperature']
categorical_features = ['Visibility', 'Weather', 'Road_Type', 'Traffic_Density', 
                        'Road_Hazards', 'Time_of_Day', 'Fatigue_Level', 'Speeding']


In [6]:
# Define preprocessor
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# TensorFlow Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(42,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Regression output
])


C:\Users\shiva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])


In [12]:
# Create the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [13]:

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [14]:
# Train the model
history = model.fit(X_train_preprocessed, y_train, validation_data=(X_test_preprocessed, y_test),
                    epochs=50, batch_size=32)

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1570.0055 - mae: 25.8242 - val_loss: 4.6371 - val_mae: 1.6241
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 44.3808 - mae: 4.7878 - val_loss: 3.0876 - val_mae: 1.2035
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 41.0441 - mae: 4.5762 - val_loss: 9.8887 - val_mae: 2.4292
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 41.3259 - mae: 4.5786 - val_loss: 1.8527 - val_mae: 0.8399
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39.1055 - mae: 4.4617 - val_loss: 7.8660 - val_mae: 2.2589
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39.8984 - mae: 4.4474 - val_loss: 1.9649 - val_mae: 0.8543
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39.6913 - mae: 4.4611 - val_loss: 2.0007 - val_mae: 0.8592
Epoch 8/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 40.0143 - mae: 4.4658 - val_loss: 1.9588 - val_mae: 0.8291
Epoch 9/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2m

In [15]:
# Evaluate the model
loss, mae = model.evaluate(X_test_preprocessed, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4114 - mae: 0.7786
Test Loss: 1.7053649425506592, Test MAE: 0.788854718208313


In [20]:
# Define the column names for the input
columns = numeric_features + categorical_features

# Wrap the input in a DataFrame
new_input = [[19, 1, 5.5, 1, 88, 22, 'Good', 'Clear', 'Rural', 'Low', 'None', 'Morning', 'Low', 'No']]
new_input_df = pd.DataFrame(new_input, columns=columns)

# Preprocess the input
new_input_processed = preprocessor.transform(new_input_df)

# Predict the risk score
predicted_risk = model.predict(new_input_processed)
print(f"Predicted Risk Score: {predicted_risk[0][0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
Predicted Risk Score: 13.007760047912598


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('vehicle_risk_model.tflite', 'wb') as file:
    file.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\shiva\AppData\Local\Temp\tmpms2jsd_o\assets


INFO:tensorflow:Assets written to: C:\Users\shiva\AppData\Local\Temp\tmpms2jsd_o\assets


Saved artifact at 'C:\Users\shiva\AppData\Local\Temp\tmpms2jsd_o'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2301262065168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2301262064592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2301262065744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2301262065360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2301262066512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2301262067664: TensorSpec(shape=(), dtype=tf.resource, name=None)
